In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

# import ensemble methods
from sklearn.ensemble import (
    BaggingClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier,
)
from xgboost import XGBClassifier

# import base estimators
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    ConfusionMatrixDisplay,
    RocCurveDisplay,
)
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import warnings

warnings.filterwarnings(
    "ignore", category=DeprecationWarning
)  # to avoid deprecation warnings


In [83]:
# Import dataset
print("Loading dataset...")
dataset = pd.read_csv("src/Data.csv")
print("...Done.")
print()

Loading dataset...
...Done.



In [84]:
# Basic statistics and dataset overview
print(f"Number of rows: {dataset.shape[0]}")
print(f"Number of columns: {dataset.shape[1]}")
print()

# Display first few rows of the dataset
print(dataset.head())
print()

# Basic statistics
data_desc = dataset.describe(include="all")
print(data_desc)
print()

# Percentage of missing values for each column
missing_values = (dataset.isnull().sum() / dataset.shape[0]) * 100
print(missing_values)

Number of rows: 10
Number of columns: 4

   Country   Age  Salary Purchased
0   France  44.0   72000        No
1    Spain  27.0   48000       Yes
2  Germany  30.0   54000        No
3    Spain  38.0   61000        No
4  Germany  40.0   69000       Yes

       Country        Age        Salary Purchased
count       10   9.000000     10.000000        10
unique       3        NaN           NaN         2
top     France        NaN           NaN        No
freq         4        NaN           NaN         5
mean       NaN  38.777778  64300.000000       NaN
std        NaN   7.693793  11681.419244       NaN
min        NaN  27.000000  48000.000000       NaN
25%        NaN  35.000000  55000.000000       NaN
50%        NaN  38.000000  64000.000000       NaN
75%        NaN  44.000000  71250.000000       NaN
max        NaN  50.000000  83000.000000       NaN

Country       0.0
Age          10.0
Salary        0.0
Purchased     0.0
dtype: float64


# Preprocessing

In [85]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Purchased"

X = dataset.drop(columns=[target_variable], axis= 1)
Y = dataset[target_variable]

print("...Done.")
print()

print("Y (Target variable):")
print(Y.head())
print("\nX (Features):")
print(X.head())

Separating labels from features...
...Done.

Y (Target variable):
0     No
1    Yes
2     No
3     No
4    Yes
Name: Purchased, dtype: object

X (Features):
   Country   Age  Salary
0   France  44.0   72000
1    Spain  27.0   48000
2  Germany  30.0   54000
3    Spain  38.0   61000
4  Germany  40.0   69000


In [86]:
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(exclude=["int64", "float64"]).columns.tolist()
print("numeric_features:")
print(numeric_features)
print("\ncategorical_features:")
print(categorical_features)

numeric_features:
['Age', 'Salary']

categorical_features:
['Country']


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)

In [88]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [89]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop="first"))
])

In [90]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
])

In [91]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
X_train = preprocessor.fit_transform(X_train)
print("X_train done")
print(X_train[0:5])
print()

# Label encoding
print("Label Encoding on train set:")
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
print("y_train done")
print(y_train[0:5])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
X_test = preprocessor.transform(X_test)
print(X_test[0:5,:])
print()

# Label encoding
print("Encoding labels...")
print(y_test[0:5])
Y_test = encoder.transform(y_test)
print("...Done")
print(Y_test[0:5])

Performing preprocessings on train set...
X_train done
[[ 1.61706195e+00  1.46885753e+00  0.00000000e+00  0.00000000e+00]
 [ 8.22715727e-01  1.09777773e+00  1.00000000e+00  0.00000000e+00]
 [-1.41104234e-15 -1.00500778e+00  0.00000000e+00  1.00000000e+00]
 [ 2.26956063e-01  8.50391200e-01  0.00000000e+00  0.00000000e+00]
 [ 4.25542617e-01  1.08231607e-01  0.00000000e+00  1.00000000e+00]]

Label Encoding on train set:
y_train done
[0 1 0 1 0]

Performing preprocessings on test set...
[[2.80858127 2.82948345 1.         0.        ]
 [2.41140816 2.33471038 0.         0.        ]]

Encoding labels...
8     No
7    Yes
Name: Purchased, dtype: object
...Done
[0 1]


# Bagging

Bagging with logistic regression as base estimator

In [92]:
# Initialize logistic regression with increased max_iter to handle convergence warnings
logistic_regression = LogisticRegression(max_iter=1000)

# Initialize BaggingClassifier with logistic regression as the base estimator
model = BaggingClassifier(estimator=logistic_regression)

# Define parameter grid for hyperparameter tuning
params = {
    "estimator__C": [0.01, 0.05, 0.1, 0.5],  # C is a hyperparameter of LogisticRegression
    "n_estimators": [5, 10, 20, 30]  # n_estimators is a hyperparameter of BaggingClassifier
}

# Set up GridSearchCV with 3-fold cross-validation
gridsearch = GridSearchCV(
    estimator=model,
    param_grid=params,
    cv=3,
    n_jobs=-1,  # Utilize all available cores for parallel processing
    verbose=2   # Increase verbosity for detailed output
)

# Perform grid search
print("Performing grid search...")
gridsearch.fit(X_train, y_train)
print("Grid search completed.")

# Display results
print(f"Best hyperparameters: {gridsearch.best_params_}")
print(f"Best cross-validation accuracy: {gridsearch.best_score_:.4f}")
print(f"Training accuracy: {gridsearch.score(X_train, y_train):.4f}")
print(f"Test accuracy: {gridsearch.score(X_test, Y_test):.4f}")

Performing grid search...
Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END ..................estimator__C=0.01, n_estimators=5; total time=   0.0s
[CV] END ..................estimator__C=0.01, n_estimators=5; total time=   0.0s
[CV] END .................estimator__C=0.01, n_estimators=10; total time=   0.0s
[CV] END ..................estimator__C=0.01, n_estimators=5; total time=   0.0s
[CV] END .................estimator__C=0.01, n_estimators=10; total time=   0.0s
[CV] END ..................estimator__C=0.05, n_estimators=5; total time=   0.0s
[CV] END .................estimator__C=0.01, n_estimators=10; total time=   0.0s
[CV] END ..................estimator__C=0.05, n_estimators=5; total time=   0.0s
[CV] END ..................estimator__C=0.05, n_estimators=5; total time=   0.0s
[CV] END .................estimator__C=0.01, n_estimators=20; total time=   0.1s
[CV] END .................estimator__C=0.01, n_estimators=30; total time=   0.0s
[CV] END .............

Bagging with decision tree as base estimator

In [93]:
# Initialisation du modèle de base et de l'algorithme d'ensemble
decision_tree = DecisionTreeClassifier()

# Initialize BaggingClassifier with decision_tree as the base estimator
model = BaggingClassifier(estimator=decision_tree)

# Grille d'hyperparamètres pour la recherche par grille
params = {
    "estimator__max_depth": [1, 2, 3],
    "estimator__min_samples_leaf": [1, 2, 3],
    "estimator__min_samples_split": [2, 3, 4],
    "n_estimators": [2, 4, 6, 8, 10],
}

# Exécution de la recherche par grille avec validation croisée
print("Running grid search...")
gridsearch = GridSearchCV(
    estimator=model,
    param_grid=params,
    cv=3, verbose=1,
    n_jobs=-1  # Parallel processing for efficiency
)
gridsearch.fit(X_train, y_train)

# Affichage des meilleurs paramètres et performances
print("Grid search complete.")
print(f"Best hyperparameters: {gridsearch.best_params_}")
print(f"Best cross-validation accuracy: {gridsearch.best_score_:.4f}")

# Évaluation des performances sur les ensembles d'entraînement et de test
train_accuracy = gridsearch.score(X_train, y_train)
test_accuracy = gridsearch.score(X_test, Y_test)
print(f"Training set accuracy: {train_accuracy:.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

Running grid search...
Fitting 3 folds for each of 135 candidates, totalling 405 fits
Grid search complete.
Best hyperparameters: {'estimator__max_depth': 1, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'n_estimators': 4}
Best cross-validation accuracy: 0.6111
Training set accuracy: 0.6250
Test set accuracy: 0.5000


/Users/marie-sophiechenevier/Library/CloudStorage/Dropbox/8-Jedha/GitHub/2-Jedha_Fullstack/.venv/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [94]:
# vérifier si X_train et X_test contiennent des valeurs manquantes ou infinies :
print("Checking for NaN or infinite values in X_train and X_test...")
print("NaN values in X_train:", np.isnan(X_train).sum())
print("NaN values in X_test:", np.isnan(X_test).sum())
print("Infinite values in X_train:", np.isinf(X_train).sum())
print("Infinite values in X_test:", np.isinf(X_test).sum())

Checking for NaN or infinite values in X_train and X_test...
NaN values in X_train: 0
NaN values in X_test: 0
Infinite values in X_train: 0
Infinite values in X_test: 0


In [95]:
# vérifier le type de données
print("Data type of X_train:", X_train.dtype)
print("Data type of X_test:", X_test.dtype)

Data type of X_train: float64
Data type of X_test: float64


# Boosting
Adaboost

Adaboost with logistic regression as base estimator

In [97]:
# Initialisation du modèle de base et de l'algorithme d'ensemble
logistic_regression = LogisticRegression(max_iter=1000)  # max_iter augmenté pour éviter les avertissements de convergence
model = AdaBoostClassifier(estimator=logistic_regression, algorithm='SAMME')  # Utilisation explicite de SAMME pour éviter l'avertissement

# Grille d'hyperparamètres pour la recherche par grille
params = {
    "estimator__C": [0.01, 0.05, 0.1, 0.5],  # 'estimator__' nécessaire pour les hyperparamètres de LogisticRegression
    "n_estimators": [5, 10, 20, 30],         # Hyperparamètre de AdaBoost pour le nombre d'estimateurs
}

# Exécution de la recherche par grille avec validation croisée
print("Running grid search...")
gridsearch = GridSearchCV(
    estimator=model,
    param_grid=params,
    cv=3,
    verbose=1,  # Affiche la progression de la recherche
    n_jobs=-1   # Utilise tous les cœurs de processeur pour accélérer la recherche
)
gridsearch.fit(X_train, y_train)

# Affichage des meilleurs paramètres et performances
print("Grid search complete.")
print(f"Best hyperparameters: {gridsearch.best_params_}")
print(f"Best cross-validation accuracy: {gridsearch.best_score_:.4f}")

# Évaluation des performances sur les ensembles d'entraînement et de test
train_accuracy = gridsearch.score(X_train, y_train)
test_accuracy = gridsearch.score(X_test, Y_test)
print(f"Training set accuracy: {train_accuracy:.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

Running grid search...
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Grid search complete.
Best hyperparameters: {'estimator__C': 0.5, 'n_estimators': 5}
Best cross-validation accuracy: 0.6111
Training set accuracy: 0.8750
Test set accuracy: 0.5000


Adaboost with decision tree as base estimator

In [98]:
# Initialisation du modèle de base et de l'algorithme d'ensemble avec SAMME pour éviter l'avertissement de dépréciation
decision_tree = DecisionTreeClassifier()
model = AdaBoostClassifier(estimator=decision_tree, algorithm='SAMME')

# Grille d'hyperparamètres pour la recherche par grille
params = {
    "estimator__max_depth": [1, 2, 3],           # Profondeur de l'arbre de décision
    "estimator__min_samples_leaf": [1, 2, 3],    # Nombre min. d'échantillons par feuille
    "estimator__min_samples_split": [2, 3, 4],   # Nombre min. d'échantillons pour diviser un nœud
    "n_estimators": [2, 4, 6, 8, 10],            # Nombre d'estimateurs dans AdaBoost
}

# Exécution de la recherche par grille avec validation croisée
print("Running grid search...")
gridsearch = GridSearchCV(
    estimator=model,
    param_grid=params,
    cv=3,
    verbose=1,  # Affiche la progression de la recherche
    n_jobs=-1   # Utilise tous les cœurs pour accélérer la recherche
)
gridsearch.fit(X_train, y_train)

# Affichage des meilleurs paramètres et performances
print("Grid search complete.")
print(f"Best hyperparameters: {gridsearch.best_params_}")
print(f"Best cross-validation accuracy: {gridsearch.best_score_:.4f}")

# Évaluation des performances sur les ensembles d'entraînement et de test
train_accuracy = gridsearch.score(X_train, y_train)
test_accuracy = gridsearch.score(X_test, Y_test)
print(f"Training set accuracy: {train_accuracy:.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

Running grid search...
Fitting 3 folds for each of 135 candidates, totalling 405 fits
Grid search complete.
Best hyperparameters: {'estimator__max_depth': 1, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'n_estimators': 2}
Best cross-validation accuracy: 0.3889
Training set accuracy: 0.6250
Test set accuracy: 0.5000


# Scikit-learn's GradientBoosting
Boosting with logistic regression as base estimator: forbidden ⛔️